In [1]:
import os
import codecs
import re
import time
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import string
from gensim.test.utils import common_texts

from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [2]:
stopwords = ["баш","без","биће","био","бити","близу","број","дана","данас","доћи","добар","добити","док","доле","дошао","други","дуж","два","често","чији","где","горе","хвала","и","ићи","иако","иде","има","имам","имао","испод","између","изнад","изван","изволи","један","једини","једном","јесте","још","јуче","кад","како","као","кога","која","које","који","кроз","мали","мањи","мисли","много","моћи","могу","мора","морао","на","наћи","наш","негде","него","некад","неки","немам","нешто","није","ниједан","никада","нисмо","ништа","њега","његов","њен","њих","њихов","око","около","она","онај","они","оно","осим","остали","отишао","овако","овамо","овде","ове","ово","о","питати","почетак","поједини","после","поводом","правити","пре","преко","према","први","пут","радије","сада","смети","шта","ствар","стварно","сутра","сваки","све","свим","свугде","тачно","тада","тај","такође","тамо","тим", "у", "учинио","учинити","умало","унутра","употребити","узети","ваш","већина","веома","видео","више","захвалити","зашто","због","желео","жели","знати"]

In [4]:
dataset_path = '../webapp/public/acts'
dataset = {}
TAG_RE = re.compile(r'<[^>]+>')
for file_path in os.listdir(dataset_path):
    with codecs.open(os.path.join(dataset_path, file_path), 'r', encoding='utf-8') as f:
        data = f.read()
        data = data.lower() #lowercase
        data = TAG_RE.sub('', data) #remove html tags
        data = data.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
        data_split = data.split() #remove extra whitespaces
        data_split = [word for word in data_split if word not in stopwords and not word.isnumeric()]
#         data = " ".join(data_split)
        dataset[file_path] = data_split

In [5]:
for key, data in dataset.items():
    print(key, data[:10])
    break

1.html ['\ufeff', 'закон', 'изгледу', 'употреби', 'грба', 'заставе', 'химне', 'републике', 'србије', '“службени']


In [6]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(list(dataset.values()))]
model = Doc2Vec(documents, vector_size=100, negative=0, window=2, min_count=1, workers=4)

In [7]:
model.infer_vector(["закон", "порез"])


array([ 0.00289631,  0.00098572,  0.00212842,  0.00487264,  0.00133219,
       -0.00445357, -0.00078048,  0.00382822,  0.00384407, -0.00290739,
        0.00207453, -0.00178721, -0.00358715,  0.00104854, -0.00362849,
       -0.00470848, -0.00283718, -0.00307637,  0.00352699, -0.00119379,
       -0.00237988,  0.00467369,  0.00020195, -0.00365505, -0.00399631,
        0.00223017,  0.00169127,  0.0028409 ,  0.00274204, -0.00353623,
       -0.00110818, -0.00173368,  0.00263991, -0.00054317, -0.00130613,
        0.00318474, -0.00186193, -0.00310255,  0.00411451, -0.00172306,
       -0.00416808,  0.00400786, -0.0021083 , -0.00473565, -0.00037968,
       -0.00300644, -0.00423395, -0.0047799 , -0.00098583, -0.00061441,
       -0.00410193,  0.00461551, -0.0025158 ,  0.00199274, -0.00168746,
       -0.00126515,  0.00468511,  0.00023907,  0.00310504,  0.00308767,
        0.00147667, -0.00017743,  0.00167538,  0.00382277,  0.00281358,
       -0.00261167, -0.00029648, -0.00032118, -0.00154835,  0.00

In [70]:
def dataset_to_vectors(dataset, model):
    for key, value in dataset.items():
        dataset[key] = model.infer_vector(value)
    X = list(dataset.values())
    return X

In [74]:
X = dataset_to_vectors(dataset, model)

In [30]:
def query_to_vec(query, model):
    query = query.lower() #lowercase
    query = TAG_RE.sub('', query) #remove html tags
    query = query.translate(str.maketrans('', '', string.punctuation + "„”")) #remove punctuation
    query_split = query.split() #remove extra whitespaces
    query_split = [word for word in query_split if word not in stopwords and not word.isnumeric()]
    docvec = model.infer_vector(query_split)
    return docvec

In [46]:
print(query_to_vec('полиција закон', model))

[-0.02886382  0.02860318 -0.0247639  -0.01801156  0.01211615  0.02293156
  0.01966118  0.02603403 -0.00643078 -0.03078434 -0.00472696  0.01638502
 -0.02721033  0.02826599 -0.0420025   0.00410564 -0.00599293 -0.06052358
  0.02349789 -0.02827974  0.01714233 -0.07583176  0.02989032 -0.02368224
  0.00377964 -0.00597609  0.02239742 -0.05422151 -0.06210215 -0.01282449
  0.07320333  0.05164192  0.02985599  0.0041664  -0.0228335  -0.011918
 -0.0091908   0.03879932 -0.01853716  0.07102756 -0.0257618  -0.00164309
  0.04848462  0.04855954 -0.04207215 -0.01406762 -0.04826201 -0.02218045
  0.00626847  0.01871544  0.00362846 -0.01597021  0.00231804  0.01276919
 -0.01902335 -0.0002609  -0.00440723 -0.01503221  0.00394467  0.02671148
  0.0474726   0.00975547  0.00765413  0.00477457 -0.01176888  0.00356766
 -0.02104325 -0.01723069  0.01882391  0.01854822  0.03280892  0.04420246
 -0.00713899 -0.03832392  0.01125211 -0.02670418  0.03143423  0.01805509
  0.02343484  0.06468233  0.01168666  0.07296823  0.0

In [47]:
def calculate_similarity_word2vec(X, model, query, top_k=10):
    """ Vectorizes the `query` via `query_to_vec` and calculates the cosine similarity of
    the `query` and `X` (all the documents) and returns the `top_k` similar documents."""

    # Vectorize the query to the same length as documents
    query_vec = query_to_vec(query, model)
    query_vec = query_vec.reshape(1,-1)
    # Compute the cosine similarity between query_vec and all the documents
    cosine_similarities = cosine_similarity(X,query_vec).flatten()
    # Sort the similar documents from the most similar to less similar and return the indices
    most_similar_doc_indices = np.argsort(cosine_similarities, axis=0)[:-top_k-1:-1]
    return (most_similar_doc_indices, cosine_similarities)

In [48]:
def show_similar_documents(df, cosine_similarities, similar_doc_indices):
    """ Prints the most similar documents using indices in the `similar_doc_indices` vector."""
    counter = 1
    for index in similar_doc_indices:
        print('Top-{}, Similarity = {}'.format(counter, cosine_similarities[index]))
        print('body: {}, '.format(df[index]))
        print()
        counter += 1

In [101]:
user_question = 'порез платити цена камата'
search_start = time.time()
sim_vecs, cosine_similarities = calculate_similarity_word2vec(X, model, user_question)
search_time = time.time() - search_start
print("search time: {:.2f} ms".format(search_time * 1000))
print()
show_similar_documents(list(dataset.keys()), cosine_similarities, sim_vecs)

search time: 12.37 ms

Top-1, Similarity = 0.3647439479827881
body: 299.html, 

Top-2, Similarity = 0.32522052526474
body: 1075.html, 

Top-3, Similarity = 0.2835598587989807
body: 47.html, 

Top-4, Similarity = 0.26462653279304504
body: 1165.html, 

Top-5, Similarity = 0.25245144963264465
body: 717.html, 

Top-6, Similarity = 0.24468496441841125
body: 611.html, 

Top-7, Similarity = 0.24096600711345673
body: 468.html, 

Top-8, Similarity = 0.23778550326824188
body: 620.html, 

Top-9, Similarity = 0.23740005493164062
body: 305.html, 

Top-10, Similarity = 0.22338607907295227
body: 1299.html, 



In [97]:
query = '''
 ЗАКОН

о давању контрагаранције Републике Србије Србији и Црној Гори по кредиту Међународног удружења за развој (Кредит за развој приватног и финансијског сектора)

„Службени гласник РС”, број 34 од 18. априла 2006.

Члан 1.

Република Србија преузима обавезу да као контрагарант измири обавезе Србије и Црне Горе, у износу од 68.100.000,00 специјалних права вучења, по основу Споразума о кредиту за развој – Кредит за развој приватног и финансијског сектора између Савезне Републике Југославије и Међународног удружења за развој, потписаног 4. јуна 2002. године, који је потврђен Законом о потврђивању Споразума о кредиту за развој – Кредит за развој приватног и финансијског сектора између Савезне Републике Југославије и Међународног удружења за развој („Службени лист СРЈ – Међународни уговори”, број 6/02).

Члан 2.

Контрагаранција из члана 1. овог закона даје се Србији и Црној Гори, на име обавезе из Закона о задужењу Савезне Републике Југославије код Међународног удружења за развој („Службени лист СРЈ”, број 36/02).

Члан 3.

Корисник кредита по основу Споразума о кредиту за развој из члана 1. овог закона је Република Србија (у даљем тексту: Корисник кредита).

Члан 4.

Средства за отплату кредита по основу Споразума о кредиту за развој из члана 1. овог закона, обезбедиће Корисник кредита.

Корисник кредита је дужан да средства за отплату кредита обезбеђује према плану отплате за сва повучена средства, у износу који укључује доспелу главницу, трошкове на одобрена средства, манипулативне трошкове кредита и остале трошкове.

Корисник кредита ће обезбеђивати средства за отплату кредита у буџету Републике Србије и вршити уплату доспелих обавеза на посебан рачун Србије и Црне Горе код Народне банке Србије, са искључивом наменом отплате кредита.

Члан 5.

Овај закон ступа на снагу осмог дана од дана објављивања у „Службеном гласнику Републике Србије”.
'''